In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import nltk
from sklearn.model_selection import train_test_split
from keras import layers
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers.recurrent import LSTM
from keras.layers import Activation, Dense
from sklearn.metrics import accuracy_score
import sklearn.metrics as metrics
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers import Dense, Dropout, Activation,Flatten
from keras import metrics, regularizers
from tensorflow.keras.optimizers import RMSprop
from sklearn import metrics

In [2]:
ddf = pd.read_csv("bin_veri_kumesi.csv", sep=";")
df = ddf[['Cumle','Sinif']]

# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['Cumle'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X = tokenizer.texts_to_sequences(df['Cumle'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Y = pd.get_dummies(df['Sinif']).values
print('Shape of label tensor:', Y.shape)

Found 7245 unique tokens.
Shape of data tensor: (1360, 250)
Shape of label tensor: (1360, 2)


In [3]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.3, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(952, 250) (952, 2)
(408, 250) (408, 2)


In [6]:
epochs = 150
batch_size = 256
model = Sequential()
model.add(Flatten())
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(Conv1D(32, 1, kernel_regularizer=regularizers.l2(0.01), activation='LeakyReLU'))
model.add(Dropout(0.2))
model.add(Conv1D(16, 1, kernel_regularizer=regularizers.l2(0.01), activation='LeakyReLU'))
model.add(Dropout(0.2))
model.add(Conv1D(4, 1, kernel_regularizer=regularizers.l2(0.01), activation='LeakyReLU'))
model.add(GlobalMaxPooling1D())
model.add(Dense(512, activation='LeakyReLU'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['acc'])
history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,verbose=1, validation_split=0.1)

accr= model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.4f}\n  Accuracy: {:0.4f}'.format(accr[0],accr[1]))

Epoch 1/150
4/4 [==============================] - 1s 150ms/step - loss: 1.4261 - acc: 0.5596 - val_loss: 1.3985 - val_acc: 0.5417
Epoch 2/150
4/4 [==============================] - 0s 99ms/step - loss: 1.3840 - acc: 0.5596 - val_loss: 1.3599 - val_acc: 0.5417
Epoch 3/150
4/4 [==============================] - 0s 95ms/step - loss: 1.3450 - acc: 0.5596 - val_loss: 1.3218 - val_acc: 0.5417
Epoch 4/150
4/4 [==============================] - 0s 98ms/step - loss: 1.3036 - acc: 0.5596 - val_loss: 1.2815 - val_acc: 0.5417
Epoch 5/150
4/4 [==============================] - 0s 119ms/step - loss: 1.2603 - acc: 0.5596 - val_loss: 1.2382 - val_acc: 0.5417
Epoch 6/150
4/4 [==============================] - 0s 101ms/step - loss: 1.2139 - acc: 0.5596 - val_loss: 1.1904 - val_acc: 0.5417
Epoch 7/150
4/4 [==============================] - 0s 98ms/step - loss: 1.1608 - acc: 0.5596 - val_loss: 1.1349 - val_acc: 0.5417
Epoch 8/150
4/4 [==============================] - 0s 98ms/step - loss: 1.0993 - acc: 0

In [5]:
y_test = [row[0] for row in Y_test]

from sklearn import metrics
y_pred=model.predict(X_test)
dene = np.argmax(y_pred, axis=1)
a = np.argmax(Y_test, axis=1)
print("F1: ",f1_score(a[:],dene[:],average='weighted'))


F1:  0.7993271188256723


y_test = [row[0] for row in Y_test]

def dene():
    sutun1 = [row[0] for row in y_pred]
    sutun2 = [row[1] for row in y_pred]
    for x in range(len(sutun1)):
        if sutun1[x] > sutun2[x]:
            sutun1[x] = 1
        else:
            sutun1[x] = 0
    return sutun1
    
from sklearn import metrics
y_pred=model.predict(X_test)
print("F1: ",f1_score(y_test[:406],dene()[:406],average='weighted'))
